In [1]:
with open("names.txt") as f:
    names = f.read().splitlines()

# if I have a very rigid dataset
names = ["lorenc", "lore", "lorec"] * 1000
# names = ["lorenc"] * 3000

# Using Pytorch

In [11]:
import torch
import torch.nn.functional as F

In [5]:
char2idx = {"#": 0}
char2idx.update({c: i + 1 for i, c in enumerate(sorted(set("".join(names).lower())))})
idx2char = {i: c for i, c in char2idx.items()}

In [6]:
char2idx

{'#': 0, 'c': 1, 'e': 2, 'l': 3, 'n': 4, 'o': 5, 'r': 6}

In [8]:
names_for_training = names[:5]
block_size = 3  # how many characters to look back

context = []
target = []
for name in names_for_training:
    tokens = ["#"] * block_size + list(name.lower()) + ["#"]
    idx = 0
    while idx + block_size < len(tokens):
        context.append([char2idx[tokens[i]] for i in range(idx, idx + block_size)])
        target.append(char2idx[tokens[idx + block_size]])
        idx += 1

print(f"{context=}")
print(f"{target=}")

context=[[0, 0, 0], [0, 0, 3], [0, 3, 5], [3, 5, 6], [5, 6, 2], [6, 2, 4], [2, 4, 1], [0, 0, 0], [0, 0, 3], [0, 3, 5], [3, 5, 6], [5, 6, 2], [0, 0, 0], [0, 0, 3], [0, 3, 5], [3, 5, 6], [5, 6, 2], [6, 2, 1], [0, 0, 0], [0, 0, 3], [0, 3, 5], [3, 5, 6], [5, 6, 2], [6, 2, 4], [2, 4, 1], [0, 0, 0], [0, 0, 3], [0, 3, 5], [3, 5, 6], [5, 6, 2]]
target=[3, 5, 6, 2, 4, 1, 0, 3, 5, 6, 2, 0, 3, 5, 6, 2, 1, 0, 3, 5, 6, 2, 4, 1, 0, 3, 5, 6, 2, 0]


In [10]:
context = torch.tensor(context, dtype=torch.int64)
target = torch.tensor(target, dtype=torch.int64)

In [12]:
inputs = F.one_hot(context, num_classes=len(char2idx))

In [14]:
inputs.shape

torch.Size([30, 3, 7])

In [16]:
embedding_matrix = torch.randn((len(char2idx), 2), dtype=torch.float32, requires_grad=True)
embedding_matrix.shape

torch.Size([7, 2])

In [19]:
encoded_inputs = inputs.float() @ embedding_matrix
encoded_inputs.shape

torch.Size([30, 3, 2])

In [21]:
W1 = torch.randn((6, 50), dtype=torch.float32, requires_grad=True)
b1 = torch.randn((50,), dtype=torch.float32, requires_grad=True)

In [23]:
output_first_layer = torch.tanh(encoded_inputs.view(-1, 6) @ W1) + b1
output_first_layer.shape

torch.Size([30, 50])

In [24]:
W2 = torch.randn((50, len(char2idx)), dtype=torch.float32, requires_grad=True)
b2 = torch.randn((len(char2idx),), dtype=torch.float32, requires_grad=True)

In [25]:
projection = (output_first_layer @ W2) + b2
projection.shape

torch.Size([30, 7])

In [30]:
# we need to apply softmax - https://en.wikipedia.org/wiki/Softmax_function
logits = projection.exp()
print(logits.shape)
count = logits.sum(dim=1, keepdim=True)
print(count.shape)
# print(count)
probs = logits / count
print(probs.shape)
# print(probs)

torch.Size([30, 7])
torch.Size([30, 1])
torch.Size([30, 7])


In [33]:
probs[0]

tensor([3.7421e-10, 1.9136e-08, 3.8777e-04, 5.5507e-03, 9.9671e-07, 9.9406e-01,
        3.6336e-10], grad_fn=<SelectBackward0>)

In [34]:
loss = -torch.log(probs[torch.arange(len(context)), target]).mean()
print(f"{loss.item()=}")

loss.item()=10.717171669006348


# Full examples

In [254]:
# names_for_training = names[:10]
names_for_training = names
context = []
target = []
for name in names_for_training:
    tokens = ["#"] + list(name.lower()) + ["#"]
    for first_token, second_token in zip(tokens, tokens[1:]):
        first_idx = char2idx[first_token]
        second_idx = char2idx[second_token]
        context.append(first_idx)
        target.append(second_idx)
print(f"{context[:10]=}")
print(f"{target[:10]=}")

context[:10]=[0, 3, 5, 6, 2, 4, 1, 0, 3, 5]
target[:10]=[3, 5, 6, 2, 4, 1, 0, 3, 5, 6]


In [255]:
len(context)

21000

In [256]:
len(target)

21000

In [257]:
context = torch.tensor(context, dtype=torch.int64)
target = torch.tensor(target, dtype=torch.int64)
W = torch.randn((len(char2idx), len(char2idx)), dtype=torch.float32, requires_grad=True)

context = F.one_hot(context, num_classes=len(char2idx))

In [258]:
for epoch in range(1_000):
    logits = context.float() @ W.float() # pick up the line, W should represent probabilities
    logits = logits.exp()
    count = logits.sum(dim=1, keepdim=True)
    probs = logits / count
    loss = -torch.log(probs[torch.arange(len(context)), target]).mean()
    print(f"{loss.item()=}")
    W.grad = None  # reset gradients
    loss.backward()
    W.data -= 1 * W.grad


loss.item()=2.9746012687683105
loss.item()=2.8236284255981445
loss.item()=2.6773669719696045
loss.item()=2.5359787940979004
loss.item()=2.3996007442474365
loss.item()=2.268334150314331
loss.item()=2.142249822616577
loss.item()=2.0213849544525146
loss.item()=1.9057458639144897
loss.item()=1.7953016757965088
loss.item()=1.6899974346160889
loss.item()=1.5897561311721802
loss.item()=1.494479775428772
loss.item()=1.4040570259094238
loss.item()=1.3183672428131104
loss.item()=1.2372866868972778
loss.item()=1.1606863737106323
loss.item()=1.0884383916854858
loss.item()=1.0204212665557861
loss.item()=0.9565091133117676
loss.item()=0.8965794444084167
loss.item()=0.8405036926269531
loss.item()=0.7881523370742798
loss.item()=0.7393856048583984
loss.item()=0.6940559148788452
loss.item()=0.652006983757019
loss.item()=0.6130728125572205
loss.item()=0.5770790576934814
loss.item()=0.54384845495224
loss.item()=0.5131967067718506
loss.item()=0.4849432706832886
loss.item()=0.45890745520591736
loss.item()=0

In [259]:
g = torch.Generator().manual_seed(42)

for _ in range(5):
    output = []
    selected = char2idx["#"]
    while True:
        one_hot = F.one_hot(torch.tensor(selected), num_classes=len(char2idx)).float()
        logits = one_hot @ W.float()  # pick up the line, W should represent probabilities
        logits = logits.exp()
        count = logits.sum()
        probs = logits / count

        selected = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        if selected == char2idx["#"]:
            break
        output.append(idx2char[selected])
    print("".join(output))


lorenc
lorenc
lorenc
lorenc
lorenc


In [260]:
logits.sum()

tensor(243.6905, grad_fn=<SumBackward0>)

In [268]:
# likelihood
# names_to_evaluate = ["lorenc"]
names_to_evaluate = names[:10]
with torch.no_grad():
    for name in names_to_evaluate:
        tokens = ["#"] + list(name.lower()) + ["#"]
        log_likelihood = 0.0
        n = 0
        for first_token, second_token in zip(tokens, tokens[1:]):
            first_idx = char2idx[first_token]
            second_idx = char2idx[second_token]

            one_hot = F.one_hot(torch.tensor([first_idx]), num_classes=len(char2idx)).float()
            logits = one_hot @ W.float()  # pick up the line, W should represent probabilities
            logits = logits.exp()
            count = logits.sum()
            probs = logits[0, second_idx] / count

            log_likelihood += torch.log(probs).mean().item()
            n += 1
    print(f"{log_likelihood=}")
    print(f"{-log_likelihood=}")
    print(f"{-log_likelihood/n=}")

log_likelihood=-0.04291695822030306
-log_likelihood=0.04291695822030306
-log_likelihood/n=0.006130994031471866


In [269]:
logits

tensor([[2.4223e+02, 9.6842e-02, 2.7559e-01, 3.1765e-01, 3.1503e-01, 1.0683e-01,
         3.4852e-01]])

In [265]:
second_idx

3